## Generate database of steady state images

In [1]:
import os
import shutil
import pickle
from itertools import product
from contextlib import suppress
import sys; sys.path.insert(0, "../..")

import numpy as np
from tqdm.auto import tqdm

import groovy.pde

In [2]:
# Create directory to store all the data
folder = 'database'
with suppress(FileNotFoundError):
    shutil.rmtree(folder)
os.mkdir(folder)

In [3]:
# Create a parameter grid
size = 3
ru_grid = np.linspace(0.7, 0.9, size)
rv_grid = np.linspace(0.1, 0.3, size)
f_grid  = np.linspace(0.034, 0.046, size)
k_grid  = np.linspace(0.061, 0.065, size)

In [4]:
# Run all the simulations
frame_t0 = groovy.pde.initialframe((100, 100), conctype='clump')

for i, (ru, rv, f, k) in enumerate(tqdm(product(ru_grid, rv_grid, f_grid, k_grid),
                                        desc='Params', leave=True, total=size**4)):
    time_array = groovy.pde.evolve(frame_t0, ru, rv, f, k, boundary='wrap')
    final_frame = time_array[:,:,-1,:]

    # Pickle the results
    with open(f"{folder}/{ru}_{rv}_{f}_{k}.pkl", 'wb') as f:
        pickle.dump(final_frame, f)

../../groovy/pde/pde_solver.py:40: RuntimeWarning: overflow encountered in square
  reactterm = frame_t1[:,:,:,0]**uexp * frame_t1[:,:,:,1]**vexp
../../groovy/pde/pde_solver.py:44: RuntimeWarning: invalid value encountered in add
  frame_t2 = frame_t1[:,:,:,0] + (diffterm - reactterm + sourceterm)*dt
../../groovy/pde/pde_solver.py:55: RuntimeWarning: invalid value encountered in add
  frame_t2 = np.stack((frame_t2, frame_t1[:,:,:,1] + (diffterm + reactterm - sinkterm)*dt), axis=-1)
../../groovy/pde/pde_solver.py:40: RuntimeWarning: overflow encountered in multiply
  reactterm = frame_t1[:,:,:,0]**uexp * frame_t1[:,:,:,1]**vexp


In [5]:
print('Expected number of files:', size**4)
print('Actual number of files:', len(os.listdir(folder)))

Expected number of files: 81
Actual number of files: 81
